In [59]:
##########################
# Load the libraries
##########################

from osgeo import gdal
import pystac
import subprocess
import rasterio
import imageio
from shapely.geometry import Polygon, mapping
from shapely.geometry import shape

import datetime
import os
import json
import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Function

In [60]:
def print_json(jsonFile):
    print(json.dumps(jsonFile, indent=2))

In [18]:
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.bottom],
            [bounds.right, bounds.top]
        ])
        
    return(bbox, mapping(footprint))

## STAC Catlog

In [61]:
###############
# The file name 
###############
file_name = "utilisation_territoire_2020_cog.tif"

In [62]:
##########################
# Define the root catalog
##########################
root_catalog = pystac.Catalog(
    id="land-use",
    description="QGIS STAC API browser extension"
)

print_json(root_catalog.to_dict())

{
  "type": "Catalog",
  "id": "land-use",
  "stac_version": "1.0.0",
  "description": "QGIS STAC API browser extension",
  "links": []
}


In [63]:
#############################
# Define Items and attributes
#############################
bbox, footprint = get_bbox_and_footprint(file_name)
item_ut_2020 = pystac.Item(
    id="utilisation_territoire_2020",
    geometry=footprint,
    bbox=bbox,
    datetime=datetime.datetime(2020,1,1),
    properties={}
)

print_json(item_ut_2020.to_dict())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "utilisation_territoire_2020",
  "properties": {
    "datetime": "2020-01-01T00:00:00Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -830370.0,
          117960.0
        ],
        [
          -830370.0,
          2091060.0
        ],
        [
          787470.0,
          117960.0
        ],
        [
          787470.0,
          2091060.0
        ],
        [
          -830370.0,
          117960.0
        ]
      ]
    ]
  },
  "links": [],
  "assets": {},
  "bbox": [
    -830370.0,
    117960.0,
    787470.0,
    2091060.0
  ],
  "stac_extensions": []
}


In [72]:
#####################################
# Add asset to item
#####################################
asset = pystac.Asset(
    href=file_name,
    media_type=pystac.MediaType.COG
)

item_ut_2020.add_asset("image", asset)

In [67]:
#####################################
# Define spatial and temporal extents
#####################################
qc_spatial_extent = pystac.SpatialExtent(bboxes=[bbox])
qc_temporal_extent = pystac.TemporalExtent(intervals=[(datetime.datetime(2012, 1, 1), datetime.datetime(2020, 1, 1))])
qc_extent = pystac.Extent(spatial=qc_spatial_extent, temporal=qc_temporal_extent)

print_json(qc_extent.to_dict())

{
  "spatial": {
    "bbox": [
      [
        -830370.0,
        117960.0,
        787470.0,
        2091060.0
      ]
    ]
  },
  "temporal": {
    "interval": [
      [
        "2012-01-01T00:00:00Z",
        "2020-01-01T00:00:00Z"
      ]
    ]
  }
}


In [69]:
####################
# Create collection
####################
qc_collection = pystac.Collection(
    id='qc',
    description="land-use for province of Quebec",
    extent=qc_extent
)
print_json(qc_collection.to_dict())

{
  "type": "Collection",
  "id": "qc",
  "stac_version": "1.0.0",
  "description": "land-use for province of Quebec",
  "links": [],
  "extent": {
    "spatial": {
      "bbox": [
        [
          -830370.0,
          117960.0,
          787470.0,
          2091060.0
        ]
      ]
    },
    "temporal": {
      "interval": [
        [
          "2012-01-01T00:00:00Z",
          "2020-01-01T00:00:00Z"
        ]
      ]
    }
  },
  "license": "proprietary"
}


In [73]:
#############################
# Add items to collection
#############################
qc_collection.add_item(item_ut_2020)

<Link rel=item target=<Item id=utilisation_territoire_2020>>

In [74]:
#####################################
# Add collections to the root catalog
#####################################
root_catalog.add_child(qc_collection)

<Link rel=child target=<Collection id=qc>>

In [75]:
item_ut_2020.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'utilisation_territoire_2020',
 'properties': {'datetime': '2020-01-01T00:00:00Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': (((-830370.0, 117960.0),
    (-830370.0, 2091060.0),
    (787470.0, 117960.0),
    (787470.0, 2091060.0),
    (-830370.0, 117960.0)),)},
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'parent', 'href': None, 'type': 'application/json'},
  {'rel': 'collection', 'href': None, 'type': 'application/json'}],
 'assets': {'image': {'href': 'utilisation_territoire_2020_cog.tif',
   'type': image/tiff; application=geotiff; profile=cloud-optimized}},
 'bbox': [-830370.0, 117960.0, 787470.0, 2091060.0],
 'stac_extensions': [],
 'collection': 'qc'}

In [77]:
os.getcwd()

'c:\\Users\\mustapha.bouhsen\\OneDrive - Habitat\\Bureau\\Projects\\stac_qgis'

In [82]:
root_catalog.normalize_and_save(root_href=os.path.join(os.getcwd(), 'land-use'), 
                           catalog_type=pystac.CatalogType.SELF_CONTAINED)